# Batch Slide label cropping

----
### Import dependancies

In [1]:
import pytesseract as pytess
import numpy as np 
import skimage 
import scipy 
import aicsimageio
import os 
import tifffile as tf
from PIL import Image
import tkinter as tk 
import pandas as pd
import matplotlib.pyplot as plt
from tkinter import filedialog

In [2]:
pytess.pytesseract.tesseract_cmd = r'C:/Users/rcorbyn/AppData/Local/Programs/Tesseract-OCR/tesseract.exe'

---- 
### Get all files

In [3]:
def get_files(im_folders):
    ''' '''

    # initalise 
    im_files = []
    #
    for root, folders, files in os.walk(im_folders):
        for file in files:
            if file[-4:] == '.png': 
                im_files.append(root + '/' +file)

    return(im_files)

----
### Get slide image

In [4]:
# # # Creates dialogue to ask directory
# # # Get the folder containing the image stack. 
root = tk.Tk()
root.attributes('-topmost', True)
root.withdraw() # Stops a second window opening
folder = filedialog.askdirectory(title = 'Select Stack file')

files = get_files(folder) 

for i in range( len(files) ):
    print(files[i])

//data.beatson.gla.ac.uk/data/RCORBYN/User_Data/Current_Projects/20241118_PlateScanner/work\37\a1e5fe5763ed9211188f9bf43f9329/labelled_R09_DT_060625_GS-006.png
//data.beatson.gla.ac.uk/data/RCORBYN/User_Data/Current_Projects/20241118_PlateScanner/work\f0\516e8749f9335a1ebbd1ce49e8d0fa/labelled_R09_DT_060625_GS-005.png
//data.beatson.gla.ac.uk/data/RCORBYN/User_Data/Current_Projects/20241118_PlateScanner/work\4d\bef3818e8a27006b1fe0206bbbb59f/labelled_R09_DT_060625_GS-008.png
//data.beatson.gla.ac.uk/data/RCORBYN/User_Data/Current_Projects/20241118_PlateScanner/work\64\6005613c9ef426db50b54978367348/labelled_R09_DT_060625_GS-019.png
//data.beatson.gla.ac.uk/data/RCORBYN/User_Data/Current_Projects/20241118_PlateScanner/work\64\53daf1dbf67ff846c3d1b0f53383f4/labelled_R09_DT_060625_GS-013.png
//data.beatson.gla.ac.uk/data/RCORBYN/User_Data/Current_Projects/20241118_PlateScanner/work\64\c194664c7e3c23e17944e415649abf/labelled_R09_DT_060625_GS-016.png
//data.beatson.gla.ac.uk/data/RCORBYN/Us

----
### Letter segmentation

In [24]:
results = []

for file in files:
   
    if file[-4:] == '.png':
        # Open up the image
        rgba_image = Image.open( file)
        im_data = rgba_image.convert('RGB')
        # Reduce the image to the lowest 
        # number of dimensions. 
        im_data = np.array(im_data)
        
        #######
        # Perform edge detection
        edges = skimage.filters.sobel( np.sum(im_data, axis = 2) )
        
        #####
        # Threshold the image to remove most of the 
        # unimportant information from the image.
        
        # Find the threshold values
        threshold_val = skimage.filters.threshold_otsu(edges)
        edges[edges < threshold_val] = 0
        
        # Fill the binary masks of the letters
        fill_string = scipy.ndimage.binary_fill_holes(edges)
        
        # Invert the image and multiply by the letter
        # mask to get only the letters in the image. 
        invert = fill_string * skimage.util.invert( im_data[:, :, 0] )
        #####
        # Create a binary image from the inverted letters
        binary = np.array(invert)
        binary[binary>0] = 1
        
        # create labels from the binary image. 
        labels = skimage.measure.label(binary)
        
        # Get properties of the letters and remaining masks. 
        region_props = pd.DataFrame( skimage.measure.regionprops_table(
                        labels, properties = {'label', 'area', 
                        'axis_minor_length', 'axis_major_length'}) )
        
        # Create a mask of zeros
        filtered_image = np.zeros(im_data.shape)
        
        # Loop around all labels. 
        for i in range(region_props.shape[0]): 
            # Find the positon of mask i in the label image. 
            mask_pos = np.where(labels == region_props['label'].iloc[i])
            # Find the maximum and minimum height and width of the masks. 
            delta_x = np.abs( np.min(mask_pos[0]) - np.max(mask_pos[0]) ) 
            delta_y = np.abs( np.min(mask_pos[1]) - np.max(mask_pos[1]) ) 
            # If the mask is too long or high, remove 
            # Also remove if the area is too small. 
            if delta_x  <= 75 and delta_y <= 75 and region_props['area'].iloc[i] > 250: 
                # keep the mask. 
                filtered_image[mask_pos[0], mask_pos[1]] = skimage.util.invert(
                                            im_data[mask_pos[0], mask_pos[1], :] )
            
        filtered_image = np.array(filtered_image, dtype = 'uint8')
        
        filtered_image[filtered_image>150] = 255
        filtered_image[filtered_image<150] = 0

        tf.imwrite('//data.beatson.gla.ac.uk/data/RCORBYN/User_Data/Current_Projects/20241118_PlateScanner/20250731_Labelled_im/' + 
                  os.path.basename(file[0:-4]) + '_filtered.png', filtered_image)
        
        text = pytess.image_to_string(filtered_image)
    
        text = text.replace( chr(10), '_' )
        text = text.replace( ' ' , '_' )
        text = text.replace( '/', '_' )
        text = text.replace('colorslide', '')
        text = text.replace('=', '')

        results.append(text)
        print(os.path.basename(file[0:-4])) 
        print( text + '\n') 

dictionary = {'Original File Name': np.array(file), 
              'Ryans OCR Code rename': np.array(results)}

results_df = pd.DataFrame(dictionary)

# results_df.to_excel('//data.beatson.gla.ac.uk/data/RCORBYN/User_Data/Current_Projects/20250703_Danis/Danis slides for relabelling/' + 
#                    'OCR_rename_test.xlsx')

labelled_R09_DT_060625_GS-006
GS_LIV_RBCMT_RMH135_2e_23_2637__SM_ROQ__25x_522_

labelled_R09_DT_060625_GS-005
GS_LIV_RBCMT_RMH135_2d_23_2635__SM_RO9Q__25x_522_

labelled_R09_DT_060625_GS-008
GS_LIV_RBCMT._RMH137_2d_23_2645__SM_RO9_25x_522_

labelled_R09_DT_060625_GS-019
SM_ROg_25x_522_

labelled_R09_DT_060625_GS-013
GS_LIV_RBCMT_RMH156_1b_24_18453__GS_RO9__25x_522_

labelled_R09_DT_060625_GS-016
GS_LIV_RBCMT_RMH156_1f_24_18452__GS_RO9_25x_522_

labelled_R09_DT_060625_GS
GS_LIV_RBCMT_RMH126_5a_23_2641__SM_ROQ__25x_522_

labelled_R09_DT_060625_GS-014
LIV_RBCMT_RMH156_1d_24_18448_GS_ROQ_25x_522_

labelled_R09_DT_060625_GS-010
GS_LIV_RBCMT_RMH153_6a_24_18454__GS_ROS_25x_522_

labelled_R09_DT_060625_GS-025
GS_LIV_RBBCMT_RMQ35_2c_21_15698__SM_ROQ__25x_522_

labelled_R09_DT_060625_GS-018
GS_LIV_RBCMT_RMH152_7a_24_21326__DT_ROS_25_x_522_

labelled_R09_DT_060625_GS-022
eS_LIV_RBBCMT_RMQ31_4a_21_9448__SM_ROS_25x_522_

labelled_R09_DT_060625_GS-001
GS_LIV_RBCMT_RMH126_5b_23_2643__SM_ROS_25x_522_
